# TF-IDF & Vector Space Model


## Apa itu VSM (Vector Space Model)


Vector Space Model (VSM) adalah kerangka kerja matriks yang digunakan dalam temu kembali informasi dan pemrosesan bahasa alami (NLP) untuk merepresentasikan dan menganalisis data visual. VSM sangat penting dalam penggalian teks, pencarian dokumen, dan tugas-tugas pembelajaran mesin berbasis teks seperti klasifikasi dokumen, pencarian informasi, dan analisis kemiripan teks.


![image.png](https://i0.wp.com/spotintelligence.com/wp-content/uploads/2023/09/vector-space-model.jpg?resize=960%2C540&ssl=1)


Setiap dimensi berhubungan dengan term yang unik, sementara dokumen dan query dapat direpresentasikan sebagai vektor di dalam ruang tersebut.


## Term Frequency-Inverse Document Frequency (TF-IDF)


TF-IDF adalah singkatan dari Term Frequency Inverse Document Frequency. Hal ini dapat didefinisikan sebagai perhitungan seberapa relevan sebuah kata dalam kumpulan atau corpus terhadap sebuah teks. Nilai relevansi meningkat secara relatif terhadap berapa kali sebuah kata muncul di dalam teks, namun dikompensasi oleh frekuensi kata di dalam corpus (kumpulan data).


## Term Frequency


Pada dokumen d, frekuensi merepresentasikan jumlah kemunculan kata t. Oleh karena itu, kita dapat melihat bahwa frekuensi akan menjadi lebih relevan ketika sebuah kata muncul dalam teks, yang mana hal ini bersifat relatif. Karena urutan istilah tidak signifikan, kita dapat menggunakan vektor untuk mendeskripsikan teks dalam kumpulan model term. Untuk setiap istilah tertentu dalam teks, ada sebuah entri dengan nilai yang merupakan frekuensi term.


**tf(t,d) = count of t in d / number of words in d**


## Inverse Document Frequency


Pada dasarnya, ini menguji seberapa relevan kata tersebut. Tujuan utama dari pencarian ini adalah untuk menemukan dokumen yang sesuai dengan pencarian. Karena tf menganggap semua istilah sama pentingnya, maka frekuensi term tidak hanya dapat digunakan untuk mengukur bobot term dalam dokumen. Pertama, cari frekuensi dokumen dari suatu istilah t dengan menghitung jumlah dokumen yang mengandung term tersebut:


**idf(t) = log(N/ df(t))**


Penjelasan:<br>
**df(t)** = **N(t)**<br>
dimana<br>
**df(t)** = **Document frequency of a term t**<br>
**N(t)** = **Number of documents containing the term t**<br>


## TF-IDF (Term Frequency-Inverse Document Frequency)


**tf-idf(t, d) = tf(t, d) \* idf(t)**


## Proses TF-IDF dan menjadikannya VSM (Vector Space Model)


### Import Library/Tool yang dibutuhkan


In [29]:
# Library untuk data manipulation
import pandas as pd
from tqdm import tqdm
import re
import string

# Library untuk text preprocessing
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

# Library untuk text vectorization/TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Library untuk save model
import pickle

[nltk_data] Downloading package stopwords to C:\Users\LAB
[nltk_data]     SISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\LAB
[nltk_data]     SISTER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


- **pandas** digunakan untuk membuat dataframe agar mudah dibaca.
- **tqdm** Untuk mentracking proses program.
- **re** (regular expression) digunakan untuk mengenali pola kata/kalimat.
- **nltk** (natural language toolkit) berfungsi untuk melakukan proses yang berkaitan dengan bahasa (teks).
- **sklearn** digunakan dalam pemrosesan data untuk kebutuhan machine learning atau data science. Dalam tugas ini, menggunakan TfidfTransformer untuk menghitung TF-IDF.
- **sastrawi** berfungsi untuk membersihkan mengurangi kata-kata imbuhan menjadi kata basic (sastrawi untuk stemming bahasa Indonesia).
- **pickle** unutk menyimpan model.


### Import data berita CSV


In [30]:
data = pd.read_csv("../tugas1/data_100.csv")
data.columns = data.columns.str.strip()
data

,judul,tanggal,isi,kategori
0,FOTO: Melihat Kehancuran Gaza usai Satu Tahun ...,"Senin, 07 Okt 2024 10:30 WIB",Satu tahun agresi Israel ke Gaza yang dimulai ...,internasional
1,"VIDEO: Setahun Agresi Israel ke Gaza, Korban T...","Senin, 07 Okt 2024 10:03 WIB",Agresi Israel ke Jalur Gaza telah memasuki sat...,internasional
2,Jenderal Brigade Al-Quds Iran Hilang usai Isra...,"Senin, 07 Okt 2024 09:59 WIB","Kepala pasukan Brigade Al-Quds Iran, Esmail Qa...",internasional
3,VIDEO: Kelompok Militan Pakistan Serang Karach...,"Senin, 07 Okt 2024 09:28 WIB",Sebuah ledakan terjadi di dekat bandara intern...,internasional
4,"Setahun Agresi Gaza, Israel Was-was Antisipasi...","Senin, 07 Okt 2024 09:25 WIB",Pasukan Pertahanan Israel (IDF) bersiaga ketat...,internasional
...,...,...,...,...
95,VIDEO: Momen Perang Yel-yel Warnai Debat Perda...,"Minggu, 06 Okt 2024 20:42 WIB",Perang yel-yel dari tiga pendukung calon guber...,nasional
96,Dharma Sindir Gagasan RK-Pramono soal Kemaceta...,"Minggu, 06 Okt 2024 20:34 WIB","Calon gubernur jalur independen, Dharma Pongre...",nasional
97,Pramono Janji Buat Jaringan Transjakarta hingg...,"Minggu, 06 Okt 2024 20:31 WIB",Calon gubernur Jakarta Pramono Anung berjanji ...,nasional
98,Jurus RK Atasi Macet: Bikin Angkutan Sungai Hi...,"Minggu, 06 Okt 2024 20:29 WIB",Calon Gubernur Jakarta Ridwan Kamil mengungkap...,nasional


Import data berita csv yang telah disimpan sebelumnya,<br>
dan membersihkan whitespace pada col header.


### Mengacak data


In [31]:
data = data.sample(frac = 1, ignore_index=True)

Mengacak data yang sebelumnya berdasarkan kategori 50:50, menjadi acak-acak.


### Fungsi clean_text()


In [32]:
def clean_text(text):
	text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', ' ', text) # Menghapus https* and www*
	text = re.sub(r'@[^\s]+', ' ', text) # Menghapus username
	text = re.sub(r'[\s]+', ' ', text) # Menghapus tambahan spasi
	text = re.sub(r'#([^\s]+)', ' ', text) # Menghapus hashtags
	text = re.sub(r'rt', ' ', text) # Menghapus retweet
	text = text.translate(str.maketrans("","",string.punctuation)) # Menghapus tanda baca
	text = re.sub(r'\d', ' ', text) # Menghapus angka
	text = text.lower()
	text = text.encode('ascii','ignore').decode('utf-8') #Menghapus ASCII dan unicode
	text = re.sub(r'[^\x00-\x7f]',r'', text)
	text = text.replace('\n','') #Menghapus baris baru
	text = text.strip()
	return text

Fungsi ini untuk membersihkan text, lebih tepatnya membersihkan teks seperti; menghapus hastag, unicode, dsb.


### Fungsi stemming_indo()


In [33]:
def stemming_indo(text):
	factory = StemmerFactory()
	stemmer = factory.create_stemmer()
	text = ' '.join(stemmer.stem(word) for word in text)
	return text

Fungsi ini digunakan untuk menstemming atau membersihkan kata seperti:

katanya = kata<br>
menggunakan = guna


### Fungsi clean_stopword()


In [34]:
def clean_stopword(tokens):
	listStopword =  set(stopwords.words('indonesian'))
	removed = []
	for t in tokens:
		if t not in listStopword:
			removed.append(t)
	return removed

Fungsi untuk membuang kata yang tidak digunakan seperti:

di, dan, dsb.


### Proses preprosesing text setiap dokumen


In [35]:
def preprocess_text(content):
	result = []
	for text in tqdm(content):
		cleaned_text = clean_text(text)
		tokens = word_tokenize(cleaned_text)
		cleaned_stopword = clean_stopword(tokens)
		stemmed_text = stemming_indo(cleaned_stopword)
		result.append(stemmed_text)
	return result

data['cleaned_text'] = preprocess_text(data['isi'])

100%|██████████| 100/100 [00:01<00:00, 51.10it/s]


Proses untuk mempersiapkan teks pada setiap dokumen yang diproses menggunakan fungsi-fungsi yang sudah dibuat sebelumnya, yang selanjutnya akan di tf-idf dan vsm.


### Proses TF-IDF dan pembuatan VSM


#### Split data


In [36]:
data_train = data[:80]
data_test = data[80:]
data_train

,judul,tanggal,isi,kategori,cleaned_text
0,"3 Negara Barat Tembak Rudal Iran ke Israel, Bi...","Minggu, 06 Okt 2024 11:08 WIB","Pemimpin Tertinggi Iran, Ayatollah Ali Khamene...",internasional,pimpin te inggi iran ayatollah ali khamenei se...
1,Dharma Pongrekun Sebut Pandemi Covid-19 Agenda...,"Minggu, 06 Okt 2024 21:36 WIB",Calon Gubernur DKI Jakarta Dharma Pongrekun me...,nasional,calon gubernur dki jaka a dharma pongrekun pan...
2,KPK Lakukan OTT di Pemprov Kalsel,"Minggu, 06 Okt 2024 21:20 WIB",Komisi Pemberantasan Korupsi (KPK) melakukan O...,nasional,komisi berantas korupsi kpk operasi tangkap ta...
3,AHY Ujian Terbuka Promosi Doktor di Unair Hari...,"Senin, 07 Okt 2024 10:01 WIB",Menteri Agraria dan Tata Ruang/Kepala Badan Pe...,nasional,menteri agraria tata ruangkepala badan pe anah...
4,Netanyahu Akui Bunuh Bos Hizbullah sampai Risi...,"Senin, 07 Okt 2024 07:11 WIB","Perdana Menteri Israel, Benjamin Netanyahu, me...",internasional,perdana menteri israel benjamin netanyahu aku ...
...,...,...,...,...,...
75,"Israel Klaim Bunuh 440 Militan Hizbullah, Hanc...","Minggu, 06 Okt 2024 05:15 WIB",Militer Israel mengklaim telah menewaskan seki...,internasional,militer israel klaim tewas juang hizbullah han...
76,"Sempat Cekcok, Macron Tegaskan ke Netanyahu Ko...","Senin, 07 Okt 2024 05:25 WIB",Presiden Prancis Emmanuel Macron menegaskan ke...,internasional,presiden prancis emmanuel macron komitmen tegu...
77,"Luncurkan 200 Rudal ke Israel, Siapa yang Paso...","Minggu, 06 Okt 2024 06:55 WIB",Iran menjadi sorotan usai meluncurkan sekitar ...,internasional,iran sorot luncur rudal israel selasa malam an...
78,FOTO: Evakuasi WNA dari Lebanon saat Serangan ...,"Minggu, 06 Okt 2024 19:27 WIB",Sejumlah negara langsung mengevakuasi warganya...,internasional,negara langsung evakuasi warga lebanon serang ...


Split data menjadi 80 data untuk train dan 20 data untuk testing dari 100 data yang ada.


#### TF-IDF & VSM


In [37]:
def tfidf_vsm(data, kategori):
	tfidf = TfidfVectorizer()
	tfidf_matrix = tfidf.fit_transform(data)
	feature_names = tfidf.get_feature_names_out()
	
	df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
	df_tfidf.insert(0, 'Kategori Berita', kategori.reset_index(drop=True))

	return tfidf, df_tfidf

tfidf_model, df_tfidf = tfidf_vsm(data_train['cleaned_text'], data_train['kategori'])

In [38]:
def model_tf_idf(data, model, kategori):
	tfidf_matrix = model.transform(data)
	feature_names = model.get_feature_names_out()
	
	df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
	df_tfidf.insert(0, 'Kategori Berita', kategori.reset_index(drop=True))

	return df_tfidf

df_tfidf_test = model_tf_idf(data_test['cleaned_text'], tfidf_model, data_test['kategori'])

In [39]:
df_tfidf_test.head()

,Kategori Berita,abad,abai,abbas,abc,abdel,abidjan,abu,acara,acerbi,...,yoon,yordania,york,young,youtube,yubileum,yudea,yudhoyono,zionis,zona
0,nasional,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,internasional,0.000000,0.000000,0.06166,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.074221,0.0
2,internasional,0.000000,0.091235,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.083367,0.0,0.0,0.0,0.054910,0.0
3,internasional,0.065643,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,internasional,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0


In [40]:
df_tfidf

,Kategori Berita,abad,abai,abbas,abc,abdel,abidjan,abu,acara,acerbi,...,yoon,yordania,york,young,youtube,yubileum,yudea,yudhoyono,zionis,zona
0,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,nasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
2,nasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
3,nasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.205246,0.000000,0.0
4,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.088283,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
76,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
77,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.037868,0.0,0.0,0.0,0.000000,0.000000,0.0
78,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


Proses untuk membuat TF-IDF dan membentuk VSM dalam dataframe.


## Save Dataset & Model


In [41]:
df_tfidf.to_csv("data_train_vsm.csv", index=False)
df_tfidf_test.to_csv("data_test_vsm.csv", index=False)

In [42]:
# with open('tfidf_model.pkl', 'wb') as f:
#     pickle.dump(tfidf_model, f)